In [11]:
# converting tsv file into dataframes
import pandas as pd
df = pd.read_csv("/train.tsv", delimiter='\t', header=None, names=['query', 'wellness'])
print('Number of training sentences: {:,}\n'.format(df.shape[0]))
df.sample(10)

Number of training sentences: 17,500



,query,wellness
14568,What are the top five jellyfishes ?,0.8
14753,Supports combustion physical of chemical prope...,0.0
8315,What was the econmic characeristics of colonia...,0.4
9960,What is the 13th amendment forbidding owning o...,0.2
10721,How much money on average do pharmacists make ?,1.0
11133,Summerian priests were what ?,0.0
9495,The meaning of air pollution ?,0.0
12350,What is mars 's distinguishing color ?,0.2
12831,Which gas is used to fill a balloon ?,1.0
9371,What is lil mama fathers name ?,0.2


In [39]:
sentences = []
for sen in df['query']:
  sentences.append(sen)

print(sentences)

# if intensity >= 0.8 then it is considered wellformed(1) query else(0) 
wellness = []
for q in df['wellness']:
    if float(q) >= 0.8:
      wellness.append(1)
    else:
      wellness.append(0)

print(wellness)

['The European Union includes how many ?', 'What are Mia Hamms accomplishment ?', 'Which form of government is still in place in greece ?', 'When was the canal de panama built ?', 'What color is the black box on commercial aeroplane ?', 'An element on the periodic table ?', 'How did samoans come into samoa ?', 'Who disscoverd rihanna ?', 'Modern day meter is defined ?', 'What is the value of military payment certificate series 661 ?', 'Adjective starts with the letter e ?', 'Population of owls just in north america ?', "What is ireland 's favorite foods ?", 'How do you remove the stereo in a hyundai elantre ?', 'When did herbert walker bush die ?', 'Huge flowers that start with letter C ?', 'What is the sound quality of the oboe ?', 'How do you become a more confident ?', 'What is the harp seal pups weight ?', 'Can you catch mew three in emerald ?', 'Is johnny depp a celtic fan ?', 'Where is the remove history ?', '2 What were the sources of the American economic recovery of the 1980s 

In [90]:
from keras.preprocessing.sequence import pad_sequences

# Padding/truncating all sentences to values 64
MAX_LEN = 64
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

In [45]:
attention_masks = []
for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)

In [46]:

from sklearn.model_selection import train_test_split
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, wellness, random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, wellness,random_state=2018, test_size=0.1)

In [47]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [50]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [51]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2, output_attentions = False, output_hidden_states = False)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [52]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [53]:
optimizer = AdamW(model.parameters(),lr = 2e-5, eps = 1e-8 )

In [54]:
from transformers import get_linear_schedule_with_warmup
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [ ]:
import numpy as np
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [56]:
import time
import datetime
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [57]:
import random
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
      
        loss = outputs[0]


        total_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)            
    
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        
        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():        

            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_accuracy += tmp_eval_accuracy

        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    493.    Elapsed: 0:00:27.
  Batch    80  of    493.    Elapsed: 0:00:55.
  Batch   120  of    493.    Elapsed: 0:01:22.
  Batch   160  of    493.    Elapsed: 0:01:49.
  Batch   200  of    493.    Elapsed: 0:02:17.
  Batch   240  of    493.    Elapsed: 0:02:44.
  Batch   280  of    493.    Elapsed: 0:03:11.
  Batch   320  of    493.    Elapsed: 0:03:39.
  Batch   360  of    493.    Elapsed: 0:04:06.
  Batch   400  of    493.    Elapsed: 0:04:33.
  Batch   440  of    493.    Elapsed: 0:05:00.
  Batch   480  of    493.    Elapsed: 0:05:28.

  Average training loss: 0.45
  Training epcoh took: 0:05:36

Running Validation...
  Accuracy: 0.82
  Validation took: 0:00:13

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    493.    Elapsed: 0:00:27.
  Batch    80  of    493.    Elapsed: 0:00:55.
  Batch   120  of    493.    Elapsed: 0:01:22.
  Batch   160  of    493.    Elapsed: 0:01:49.
  Batch   200  of    493.    Elap

In [59]:
# data preperation for test set.
import pandas as pd
df = pd.read_csv("/test.tsv", delimiter='\t', header=None, names=['query', 'wellness'])
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

sentences = []
for sen in df['query']:
  sentences.append(sen)

wellness = []
for q in df['wellness']:
    if float(q) >= 0.8:
      wellness.append(1)
    else:
      wellness.append(0)


input_ids = []

for sent in sentences:
    encoded_sent = tokenizer.encode(sent, add_special_tokens = True)
    
    input_ids.append(encoded_sent)


input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []

for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(wellness)


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 3,850



In [91]:
# test set evaluation

model.eval()

predictions , true_labels = [], []

for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids, b_input_mask, b_labels = batch
  
  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

  logits = outputs[0]

  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  predictions.append(logits)
  true_labels.append(label_ids)


In [75]:
from sklearn.metrics import matthews_corrcoef
matthews_set = []

print('Calculating Matthews Corr. Coef. for each batch...')

for i in range(len(true_labels)):
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()  
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
  matthews_set.append(matthews)

Calculating Matthews Corr. Coef. for each batch...


In [77]:
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]

mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('MCC: %.3f' % mcc)

MCC: 0.620


In [83]:
# calculating accuracy and f1 score
from sklearn.metrics import f1_score,accuracy_score
acc_score = accuracy_score(flat_true_labels,flat_predictions)
print("Accuracy Score: {}".format(acc_score))
f1 = f1_score(flat_true_labels, flat_predictions)
print("F1 Score: {}".format(f1))

Accuracy Score: 0.8067532467532468
F1 Score: 0.775226586102719
